<div align='center'>

# OVERVIEW
</div>

This notebook aims to clarify whether or not any of the suggested Payments & Transfer services are acceptable for use for our Customer Support Center system, as defined in the [`customer_support_rep_persona_01.md` persona document](../../customer_rep_persona_store/customer_support_rep_persona_01.md).

The suggested services are listed below:

1. Etherscan (Ethereum Sepolia Testnet)
2. MetaMask

The criteria for qualifying a suggestion as a tool is defined below:

1. Python-based.
2. Open-source (free, avoid freemium as much as possible) customer-facing and producer-facing access.
3. Rate limit restrictions allow for appropriate edge-case testing.
4. Supports core architectural choice for LangChain ecosystem (e.g. supports asynchronous backends, etc).
5.  I can currently access the tool as a human (i.e. confirmation that it is operable as a human, and has not been deprecated/restricted discreetly).
6. I can perform its expected basic operations.

## Pre-Notebook Initialization

1. Setup MetaMask (Google Chrome browser extension).
2. Elicit SepoliaETH from a Sepolia faucet.
3. Verify transaction on [Sepolia Etherscan](sepolia.etherscan.io).

<u>Notes</u>:
- Ethereum addresses are the same across all EVM networks.

In [5]:
# Required Modules

import os
import requests
from dotenv import load_dotenv
load_dotenv()

True

## Etherscan

<ol>
<li> Python-based: Available
<li> Open-source: Free

<li> Maximum Rate Limit Restriction:
<ul>
<li> 5 calls per second.
<li> 100K calls per day.
<li> 3 API keys per account.
</ul>

<li> Supports Core Architectural Choice
<li> Service is confirmed to still be active & accessible
</ol>

### Expected Basic Operations

1. [Get Native Balance for an Address](https://docs.etherscan.io/api-reference/endpoint/balance).
2. [Get Normal Transactions By Address](https://docs.etherscan.io/api-reference/endpoint/txlist).
3. [Get ERC20 Token Transfers by Address](https://docs.etherscan.io/api-reference/endpoint/tokentx).
4. [Get Address Funded By](https://docs.etherscan.io/api-reference/endpoint/fundedby).
5. [Get Event Logs by Address](https://docs.etherscan.io/api-reference/endpoint/getlogs).
6. [Get ERC20-Token Account Balance for TokenContractAddress](https://docs.etherscan.io/api-reference/endpoint/tokenbalance).

**<u>Notes:</u>**
1. Etherscan offers a JSON-RPC API for interacting with its system.

### Setup

Etherscan uses an Remote Procedure Call (RPC) API - it is not RESTful.

In [6]:
# Class Definition for API calls

class Simple_Etherscan_RPC_API_Client():

    def __init__(
                self, 
                 random_eth_address: str = "0xC257274276a4E539741Ca11b590B9447B26A8051" # Randomly selected (via Google Search) for service confirmation purposes.
                 ):
        
        self.random_eth_address = random_eth_address
        self.SUCCESS_MSG = "[INFO]: Your GET request was been performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        self.data = None

    def perform_get_request(self, params: dict):
        """Performs GET request"""

        try:
            if "apikey" not in params.keys():
                params["apikey"] = os.getenv("ETHERSCAN_API_KEY")

            response = requests.get(
                url="https://api.etherscan.io/v2/api",
                params=params
            )

            self.data = response.json()
            
            print(self.SUCCESS_MSG)

        except Exception as e:
            print(self.ERROR_MSG, e)
            return 0

        return (f"Value for field 'result' is: {response.json()['result']}")

### 1. Get Native Balance for an Address

Retrieves the native token balance held by a specific address.

In [7]:
# Client 1 Init

etherscan_client1 = Simple_Etherscan_RPC_API_Client()

# Making API call

params1 = {
    "chainid":1,
    "address": etherscan_client1.random_eth_address,
    "module":"account",
    "action": "balance",
    "tag": "latest"
                }

etherscan_client1.perform_get_request(params1)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: 84479637358432397"

In [8]:
# Converting WEI to ETH

wei = int(etherscan_client1.data['result'])
eth = wei / 1018
print(f"Total ETH Amount: {eth}")

Total ETH Amount: 82985891314766.6


### 2. Get Normal Transactions By Address

Retrieves the transaction history of a specified address, with optional pagination.

In [9]:
# Client 2 Init
etherscan_client2 = Simple_Etherscan_RPC_API_Client()


params2 = {
    "chainid":"1",
    "module": "account",
    "action": "txlist",
    "address": etherscan_client2.random_eth_address,
    "startblock": 0,
    "endblock": 9999999999,
    "page": 1,
    "offset": 1,
    "sort": "desc"
}

etherscan_client2.perform_get_request(params2)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: [{'blockNumber': '23883863', 'timeStamp': '1764171563', 'hash': '0xc33d26c762c6527c300526abbfa092849d87595cc5609bc2f6ef853333b9d48c', 'nonce': '7', 'blockHash': '0x0790935e7f7fd1a25dd8beda47bcdeb0f341f60bdcdf8661fa6c4bd78c3e09fd', 'transactionIndex': '149', 'from': '0xd1bd16dba8daca7923eb064dd2be29afc115e87e', 'to': '0xc257274276a4e539741ca11b590b9447b26a8051', 'value': '265815149418', 'gas': '21000', 'gasPrice': '83014813', 'isError': '0', 'txreceipt_status': '1', 'input': '0x', 'contractAddress': '', 'cumulativeGasUsed': '31047601', 'gasUsed': '21000', 'confirmations': '59009', 'methodId': '0x', 'functionName': ''}]"

### 3. Get ERC20 Token Transfers by Address

Retrieves the list of ERC-20 token transfers made by a specified address, with optional filtering by token contract.

In [10]:
# Client 3 Init

etherscan_client3 = Simple_Etherscan_RPC_API_Client()

params3 = {
    "chainid": "1",
    "module": "account",
    "action": "tokentx",
    "contractaddress": "0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2",
    "address": "0x4e83362442b8d1bec281594cea3050c8eb01311c",
    "startblock": 0,
    "endblock": 9999999999,
    "page": 1,
    "offset": 1,
    "sort": "desc"
}

etherscan_client3.perform_get_request(params3)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: [{'blockNumber': '15572639', 'timeStamp': '1663652063', 'hash': '0x62afa8a25e569f9659e35856284e8a674fadb3029f522d0556ac6ec01adb38ad', 'nonce': '1579', 'blockHash': '0x530b533d29e73ca8376eb1298199f52fa0f34d43f2d381febaf02312453e19ec', 'from': '0x4e83362442b8d1bec281594cea3050c8eb01311c', 'contractAddress': '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2', 'to': '0xe8c6c9227491c0a8156a0106a0204d881bb7e531', 'value': '163529846812879587', 'tokenName': 'Maker', 'tokenSymbol': 'MKR', 'tokenDecimal': '18', 'transactionIndex': '62', 'gas': '254396', 'gasPrice': '6650574109', 'gasUsed': '181367', 'cumulativeGasUsed': '4085175', 'input': 'deprecated', 'methodId': '0x5ae401dc', 'functionName': 'multicall(uint256 deadline, bytes[] data)', 'confirmations': '8370233'}]"

### 4. Get Address Funded By

Retrieves the address and transaction that first funded a specific address, useful for tracing fund origins.

In [11]:
# Client 4 Init
etherscan_client4 = Simple_Etherscan_RPC_API_Client()

params4 = {
    "chainid":"1",
    "module":"account",
    "action": "fundedby",
    "address": "0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97"
}

etherscan_client3.perform_get_request(params4)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: {'block': 16976894, 'timeStamp': '1680628103', 'fundingAddress': '0x21a31ee1afc51d94c2efccaa2092ad1028285549', 'fundingTxn': '0xb5b9e8281c99d88258931da08a67650a40684d9d81d3c01d759b1442a0734e85', 'value': '300000000000000000'}"

### 5. Get Event Logs by Address

Retrieves event logs from a specific address, with optional block range filtering.

In [12]:
# Client 5 Init
etherscan_client5 = Simple_Etherscan_RPC_API_Client()

params5 = {
    "chainid": "1",
    "module": "logs",
    "action": "getLogs",
    "address": "0xbd3531da5cf5857e7cfaa92426877b022e612cf8",
    "fromBlock": 12878196,
    "toBlock": 12878196,
    "page": 1,
    "offset": 1000
}

etherscan_client5.perform_get_request(params5)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: [{'address': '0xbd3531da5cf5857e7cfaa92426877b022e612cf8', 'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x0000000000000000000000000000000000000000000000000000000000000000', '0x000000000000000000000000c45a4b3b698f21f88687548e7f5a80df8b99d93d', '0x00000000000000000000000000000000000000000000000000000000000000b5'], 'data': '0x', 'blockNumber': '0xc48174', 'blockHash': '0x837e109ab8b1b40ec7d1032bff82397325d85e719b97d900fa0d9aa9745b2c27', 'timeStamp': '0x60f9ce56', 'gasPrice': '0x2e90edd000', 'gasUsed': '0x247205', 'logIndex': '0x', 'transactionHash': '0x4ffd22d986913d33927a392fe4319bcd2b62f3afe1c15a2c59f77fc2cc4c20a9', 'transactionIndex': '0x'}, {'address': '0xbd3531da5cf5857e7cfaa92426877b022e612cf8', 'topics': ['0x645f26e653c951cec836533f8fe0616d301c20a17153debc17d7c3dbe4f32b28', '0x00000000000000000000000000000000000000000000000000000000000000b5'], 'data': '0x', 'blockNumber': '0xc48174', 'blockHash': '0x837e109ab8b1b40e

### 6. Get ERC20-Token Account Balance for TokenContractAddress

Retrieves the current ERC-20 token balance for a specified address.

In [13]:
# Client 6 Init

etherscan_client6 = Simple_Etherscan_RPC_API_Client()

params6 = {
    "chainid": "1",
    "module": "account",
    "action": "tokenbalance",
    "contractaddress": "0x57d90b64a1a57749b0f932f1a3395792e12e7055",
    "address": "0xe04f27eb70e025b78871a2ad7eabe85e61212761",
    "tag": "latest"
}

etherscan_client6.perform_get_request(params=params6)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: 135499"

## MetaMask

### <u>Pre-requisite Information</u>

MetaMask has partnered with Infura to offer a comprehensive set of services to assist with dApp and Snap development, including JSON-RPC APIs for easy access to key networks, and REST APIs aimed at automating and optimizing essential development tasks. 


<ol>
<li> Python-based: <b>Available</b>
<li> Open-source: <b>Free</b>

<li> Rate Limit Restrictions (dependent on if the request is I) <a href="https://docs.metamask.io/services/concepts/websockets#pricing">a <b>WebSocket streaming</b> request</a>, II) <a href="https://docs.metamask.io/services/how-to/make-batch-requests">a <b>batch</b> request</a>, or III) <a href="https://docs.metamask.io/services/concepts/archive-data">an <b>archive</b> request</a>):

<ol type="i">
<li> Credit costs are broken down further in the following <a href="https://docs.metamask.io/services/get-started/pricing/credit-cost/">link</a>.

<ol type="a">
<li> <b>Free Tier</b>: 3M Daily Credits, 500 credits per second rate limit.
<li> <b>Developer Tier</b>: 15M Daily Credits, 4K credits per second rate limit.
<li> <b>Team Tier</b>: 75M Daily Credits, 40K credits per second rate limit.
<li> <b>Custom</b>: (Contact MetaMask Team)
</ol>

</ol>

<li> Supports core architectural choice.
<li> Service is confirmed to <b>still be active & accessible</b>.
</ol>

### <u>Expected Basic Operations</u>

1. [Send Transactions](https://docs.metamask.io/services/tutorials/ethereum/send-a-transaction/send-a-transaction-py).
2. [Get Normal Transactions By Address](https://docs.etherscan.io/api-reference/endpoint/txlist).
3. [Get ERC20 Token Transfers by Address](https://docs.etherscan.io/api-reference/endpoint/tokentx).
4. [Get Address Funded By](https://docs.etherscan.io/api-reference/endpoint/fundedby).
5. [Get Event Logs by Address](https://docs.etherscan.io/api-reference/endpoint/getlogs).
6. [Get ERC20-Token Account Balance for TokenContractAddress](https://docs.etherscan.io/api-reference/endpoint/tokenbalance).

**<u>Notes:</u>**
<ol>

<!-- 1 -->
<li> MetaMask partners with Infura to deliver a comprehensive set of services to facilitate dApp and Snap development, including i) <b>JSON-RPC APIs</b> for easy access to key networks, and ii) <b>REST APIs</b> aimed at automating and optimizing essential development tasks e.g. the <a href='https://docs.metamask.io/services/reference/gas-api'>Gas API</a>.

<!-- 2 -->
<li> The following process is used to determine the credit cost for each request:

<ul>
<li> Initial check for a specific credit value.
<li> Fallback to the Ethereum method's credit value.
<li> Fallback to the default credit value.
</ul>

<!-- 3 -->
<li> Requests that return HTTP status code errors have their own credit costs:

<ul>
<li> <code>429</code> errors: These are not charged and can occur if you've exceeded your allowed throughput limit (requests per second).
<li> <code>402</code> errors: These are not charged and can occur if you've exceeded your allowed daily credit usage.
<li> <code>4xx</code> errors: These are errors caused by human input, and consume 5 credits.
<li> <code>5xx</code> errors: These are server errors, and do not consume any credit charges.
</ul>

<!-- 4 -->
<li> Daily credit quota limits are reset every 00:00 UTC.

</ol>

### <u>Notes</u>

After a conversation with MetaMask's FinAI, I have understood MetaMask's role in this to indeed be **customer-facing, but more of a manual way of managing one's account** rather than a programmatic way - I will maintain this method for such and rather focus on familiarizing myself with it in a manual manner.

## Web3.py

After exploration, we have discovered that:

- Etherscan serves as our blockchain explorer (read-only access).
- Sepolia Testnet serves as our transaction network.

We still need a framework for programmatically interacting with the blockchain and the `web3` module seems to be the perfect solution. Despite Etherscan being our tool for read-only access of the system, it is still not entirely fit for purpose as it is post-processed, is rate-limited, and does not allow one to subscribe for events. The `web3` module allows one to read directly from the blockchain node, subscribe for blockchain events, amongst other important functions hence the need for this module to have a separate section of its own.


### <u>Service Assessment Summary</u>

<ol>
<li> Python-based: <b>Available</b>
<li> Open-source: <b>Free</b>

<li> Maximum Rate Limit Restriction (WebSocket/HTTPS/IPC):
<ul>
<li> <b>5 calls per second</b>.
<li> <b>100K calls per day</b>.
<li> <b>3 API keys per account</b>.
</ul>

<li> <b>Supports Core Architectural Choice</b>.
<li> <b>Service is confirmed to still be active & accessible</b>.
</ol>

### <u>Expected Basic Operations</u>

1. [Ethereum Address Validation Check](https://web3py.readthedocs.io/en/latest/web3.main.html#addresses).
2. [EIP55 Checksum Address Validation Check](https://web3py.readthedocs.io/en/latest/web3.main.html#web3.Web3.is_checksum_address).
3. [Convert to EIP55 Checksum Address](https://web3py.readthedocs.io/en/latest/web3.main.html#web3.Web3.to_checksum_address).
4. [Get Transaction Receipt](https://web3py.readthedocs.io/en/latest/web3.eth.html#web3.eth.Eth.get_transaction_by_block).
5. [Send Transaction](https://web3py.readthedocs.io/en/latest/web3.eth.html#web3.eth.Eth.send_transaction).


**<u>Notes:</u>**
<ol>

<!-- 1 -->
<li> The <code>web3</code> module makes use of <b>Providers/Transport Protocols</b> for connecting to a blockchain, such as:
<ul>
<li> <code><b>HTTPProvider</b></code> for connecting to HTTP and HTTPS-based JSON-RPC servers (prefixed with '(Async).+' for asynchronous support).
<li> <code><b>IPCProvider</b></code> for connecting to IPC socket based JSON-RPC servers (prefixed with '(Async).+' for asynchronous support and enforced using a persistent connection).
<li> <code><b>WebSocketProvider</b></code> for connecting to websocket based JSON-RPC servers asynchronously via a persistent connection.
</ul>

<!-- 2 -->
<li> Access to a Provider must be setup before engaging a blockchain; there are three ways to go about this:

<ul>
<li> Test Provider: Install via <code>pip install web3[tester]</code>, then instantiate as <code>EthereumTestProvider</code>.
<li> Local Provider: <a href=''>Hardware Requirements</a> are steep, but this is the safest mode of interaction. Geth client is the most popular, exposing <b>port 8545</b> to serve HTTP requests and <b>port 8546</b> for websocket requests.
<li> Remote Provider: The <b>quickest way</b> to interact with the Ethereum blockchain - this is done by specifying the endpoint (just like with a local provider).
</ul>

<!--3 -->
<li> I've learnt that Ethereum addresses can present themselves in three different formats:
<ul>
<li> Lowercase hexadecimal (with or without the '0x' (without is preferred for <b>compactness</b>)): Valid raw hex addresses that contain only numbers and lowercase letters a-f, usually used be node logs/low-level systems.
<li> Uppercase hexadecimal: Valid but <b>not recommended</b>.
<li> Mixed-case EIP-55 Checksum Format: This is the checksummed format - it encodes checksum information defined by EIP-55 (its casing is not arbitrary) used by wallets like MetaMask.
</ul>

<!--4 -->
<li> The <code>web3</code> framework inherits from Etherscan's API, extending it with its own native APIs to speed up development.

<!--5 -->
<li> One provider is supported per instance, and if there is need for more than one provider you would have to create and configure a new web3 instance per connection.

</ol>

### **Setup**

We will first begin by setting up the Test Provider before checking out the other Providers (**IPC**, **HTTP**, and **WebSocket**).

In [24]:
from web3 import Web3, EthereumTesterProvider
import os
from dotenv import load_dotenv

w3 = Web3(EthereumTesterProvider())
w3.is_connected()

load_dotenv(dotenv_path=os.path.join("..//..//.env"))

True

In [15]:
random_eth_address = "0x416299aade6443e6f6e8ab67126e65a7f606eef5"
random_eth_address

'0x416299aade6443e6f6e8ab67126e65a7f606eef5'

### 1. Ethereum Address Validation Check

In [16]:
w3.is_address(random_eth_address)

True

### 2. EIP55 Checksum Address Validation Check

In [17]:
w3.is_checksum_address(random_eth_address)

False

### 3. Convert to EIP55 Checksum Address

In [18]:
new_checksum_address = w3.to_checksum_address(random_eth_address)
new_checksum_address

'0x416299AAde6443e6F6e8ab67126e65a7F606eeF5'

In [19]:
# EIP55 Checksum Address Validation Check

w3.is_checksum_address(new_checksum_address)

True

### 4. Get Transaction Receipt

In [20]:
random_tx_hash = "0x4fa75f36d8820dacabf291e09a4f78ab9f6eb9f9e8e74004f5715d11b3e12160"

w3.eth.get_transaction_receipt(random_tx_hash)

TransactionNotFound: Transaction with hash: '0x4fa75f36d8820dacabf291e09a4f78ab9f6eb9f9e8e74004f5715d11b3e12160' not found.

### Connecting to Real Ethereum Networks

The `EthereumTesterProvider` doesn't have access to real blockchain data. To query real transactions, we need to connect to:

1. **HTTP Provider** (Recommended - easiest setup via Infura/Alchemy)
2. **IPC Provider** (Requires local Geth node - resource intensive)
3. **WebSocket Provider** (For real-time subscriptions)

#### Recommended: HTTP Provider with Infura

1. Create a free account at [infura.io](https://infura.io)
2. Create a new project and copy your Project ID
3. Add `INFURA_PROJECT_ID` to your `.env` file

In [ ]:
# Option 1: HTTP Provider with Infura (RECOMMENDED)
# Get your free Project ID from https://infura.io

INFURA_PROJECT_ID = os.getenv("INFURA_PROJECT_ID")

INFURA_SEPOLIA_URL = f"https://sepolia.infura.io/v3/{INFURA_PROJECT_ID}"
INFURA_ETHMAINNET_URL =f"https://mainnet.infura.io/v3/{INFURA_PROJECT_ID}"

# Connect to Ethereum Mainnet via Infura
if INFURA_PROJECT_ID:
    w3_mainnet = Web3(Web3.HTTPProvider(INFURA_ETHMAINNET_URL))
    print(f"Connected to Mainnet: {w3_mainnet.is_connected()}")
    print(f"Current Block Number: {w3_mainnet.eth.block_number}")
else:
    print("[WARNING]: INFURA_PROJECT_ID not found in .env file")
    print("Please add: INFURA_PROJECT_ID=your_project_id_here")
    w3_mainnet = None

Connected to Mainnet: True
Current Block Number: 23947908


In [ ]:
# Option 2: IPC Provider (requires local Geth node)
# Only use this if you have Geth running and fully synced

"""
To use IPC Provider on Windows:
1. Install Geth: https://geth.ethereum.org/downloads/
2. Run Geth: geth --syncmode "snap" --http --http.api "eth,net,web3"
3. Wait for sync to complete (can take days for full sync)
4. The IPC file will be at: \\.\pipe\geth.ipc
"""

try:
    # Windows IPC path (use raw string)
    w3_IPC = Web3(Web3.IPCProvider(r"\\.\pipe\geth.ipc"))
    if w3_IPC.is_connected():
        print(f"Connected to local Geth node via IPC")
        print(f"Current Block: {w3_IPC.eth.block_number}")
    else:
        print("[INFO]: IPC Provider not connected - Geth may not be running")
        w3_IPC = None
except Exception as e:
    print(f"[INFO]: IPC Provider unavailable: {e}")
    print("This is expected if Geth is not running locally.")
    w3_IPC = None

False

In [28]:
# Query a real transaction from Ethereum Mainnet
# Using a known transaction hash from Mainnet

mainnet_tx_hash = "0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060"  # First ever DAO hack tx

if w3_mainnet and w3_mainnet.is_connected():
    try:
        tx_receipt = w3_mainnet.eth.get_transaction_receipt(mainnet_tx_hash)
        print(f"Transaction found!")
        print(f"  Block Number: {tx_receipt['blockNumber']}")
        print(f"  From: {tx_receipt['from']}")
        print(f"  To: {tx_receipt['to']}")
        print(f"  Gas Used: {tx_receipt['gasUsed']}")
        print(f"  Status: {'Success' if tx_receipt.get('status', 1) == 1 else 'Failed'}")
    except Exception as e:
        print(f"[ERROR]: {e}")
else:
    print("[INFO]: Skipping - No mainnet connection available")
    print("Add INFURA_PROJECT_ID to your .env file to enable this")

Transaction found!
  Block Number: 46147
  From: 0xA1E4380A3B1f749673E270229993eE55F35663b4
  To: 0x5DF9B87991262F6BA471F09758CDE1c0FC1De734
  Gas Used: 21000
  Status: Success


In [29]:
# Get full transaction details
if w3_mainnet and w3_mainnet.is_connected():
    try:
        tx = w3_mainnet.eth.get_transaction(mainnet_tx_hash)
        print(f"Full Transaction Details:")
        print(f"  Block Number: {tx['blockNumber']}")
        print(f"  Transaction Index: {tx['transactionIndex']}")
        print(f"  From: {tx['from']}")
        print(f"  To: {tx['to']}")
        print(f"  Value: {w3_mainnet.from_wei(tx['value'], 'ether')} ETH")
        print(f"  Gas: {tx['gas']}")
        print(f"  Gas Price: {w3_mainnet.from_wei(tx['gasPrice'], 'gwei')} Gwei")
        print(f"  Nonce: {tx['nonce']}")
    except Exception as e:
        print(f"[ERROR]: {e}")
else:
    print("[INFO]: Skipping - No mainnet connection available")

Full Transaction Details:
  Block Number: 46147
  Transaction Index: 0
  From: 0xA1E4380A3B1f749673E270229993eE55F35663b4
  To: 0x5DF9B87991262F6BA471F09758CDE1c0FC1De734
  Value: 3.1337E-14 ETH
  Gas: 21000
  Gas Price: 50000 Gwei
  Nonce: 0


### 5. Send Transaction

In [30]:
# Send Transaction using Web3.py
# Note: This requires a funded account on the connected network

# Using EthereumTesterProvider for demonstration (creates test accounts with test ETH)
w3_test = Web3(EthereumTesterProvider())

# Get test accounts (EthereumTesterProvider creates accounts with test ETH)
accounts = w3_test.eth.accounts
sender = accounts[0]
receiver = accounts[1]

print(f"Sender Address: {sender}")
print(f"Receiver Address: {receiver}")
print(f"Sender Balance (before): {w3_test.from_wei(w3_test.eth.get_balance(sender), 'ether')} ETH")
print(f"Receiver Balance (before): {w3_test.from_wei(w3_test.eth.get_balance(receiver), 'ether')} ETH")


Sender Address: 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf
Receiver Address: 0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF
Sender Balance (before): 1000000 ETH
Receiver Balance (before): 1000000 ETH


In [31]:
# Build and send the transaction

tx_hash = w3_test.eth.send_transaction({
    'from': sender,
    'to': receiver,
    'value': w3_test.to_wei(1, 'ether'),  # Send 1 ETH
    'gas': 21000,  # Standard gas limit for ETH transfer
    'gasPrice': w3_test.eth.gas_price
})

print(f"Transaction Hash: {tx_hash.hex()}")


Transaction Hash: 7f8111e32444bab6559fd567749745278affd088e9b170e8692fff063d7fe05e


In [32]:
# Get transaction receipt and verify balances after transaction

tx_receipt = w3_test.eth.get_transaction_receipt(tx_hash)

print(f"Transaction Status: {'Success' if tx_receipt['status'] == 1 else 'Failed'}")
print(f"Block Number: {tx_receipt['blockNumber']}")
print(f"Gas Used: {tx_receipt['gasUsed']}")
print(f"\nSender Balance (after): {w3_test.from_wei(w3_test.eth.get_balance(sender), 'ether')} ETH")
print(f"Receiver Balance (after): {w3_test.from_wei(w3_test.eth.get_balance(receiver), 'ether')} ETH")


Transaction Status: Success
Block Number: 1
Gas Used: 21000

Sender Balance (after): 999998.999979 ETH
Receiver Balance (after): 1000001 ETH


### <u>Notes on Send Transaction</u>

For real-world transactions on Sepolia Testnet or Mainnet, you would need to:

1. **Use a Remote Provider** (e.g., Infura, Alchemy) instead of EthereumTesterProvider:
```python
w3 = Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/YOUR_PROJECT_ID'))
```

2. **Sign the transaction** with a private key:
```python
signed_tx = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
```

3. **Wait for confirmation**:
```python
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
```

The EthereumTesterProvider is used here for demonstration purposes as it provides pre-funded test accounts without requiring actual testnet ETH.


---

## Summary

This service assessment has evaluated the following tools for Payments & Transfers operations:

| Service | Assessment | Status |
|---------|-----------|--------|
| **Etherscan** | ✅ Python-based, Free, Rate limits acceptable | **Approved** |
| **MetaMask** | ✅ Customer-facing wallet (manual operations) | **Approved (Manual)** |
| **Web3.py** | ✅ Python-based, Free, Full blockchain access | **Approved** |

### Tool Responsibilities:
- **Etherscan**: Blockchain explorer for read-only access (transaction history, balances, event logs)
- **MetaMask**: Customer-facing wallet management (manual account operations)
- **Web3.py**: Programmatic blockchain interaction (sending transactions, subscribing to events, direct node access)

All tools meet the defined criteria and are suitable for integration into the Customer Support Center system.
